In [87]:
import cv2 as cv
import numpy as np

query = cv.imread('Trillium_s.jpg', 0)
template = cv.imread('Trillium_t.jpg', 0)

width, height = template.shape[::-1]

# Apply template matching using TM_SQDIFF
res = cv.matchTemplate(query, template, cv.TM_SQDIFF)
min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)

# If method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
top_left = min_loc
bottom_right = (top_left[0] + width, top_left[1] + height)
cv.rectangle(query, top_left, bottom_right, 255, 2)
cv.imwrite('template_match_tm_sqdiff.jpg', query)

# Using TM_CCORR
query2 = cv.imread('Trillium_s.jpg', 0)
res2 = cv.matchTemplate(query2, template, cv.TM_CCORR)
min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res2)
# Get the top-left corner of the best match
top_left = max_loc
bottom_right = (top_left[0] + width, top_left[1] + height)
cv.rectangle(query2, top_left, bottom_right, 255, 2)
cv.imwrite('template_match_tm_ccorr.jpg', query2)

# Resize and rotate query image matching
query3 = cv.imread('Trillium_s.jpg', 0)
q_height, q_width = query3.shape[0:2]
resized_query = cv.resize(query3, (q_width*2, q_height*2), interpolation =cv.INTER_AREA)
resized_q_height, resized_q_width = resized_query.shape[0:2]
rotationMatrix = cv.getRotationMatrix2D((resized_q_width/2, resized_q_height/2), float(30), 1)
rotated_query = cv.warpAffine(resized_query, rotationMatrix, (resized_q_width, resized_q_height))

res3 = cv.matchTemplate(rotated_query, template, cv.TM_CCORR)
min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res3)
top_left = max_loc

# 1. Draw not rotated rect, but out of object
#bottom_right = (top_left[0] + width*2, top_left[1] + height*2)
#cv.rectangle(rotated_query, top_left, bottom_right, 255, 2)

# 2. Draw rotated rect from top_left position
x = top_left[0]
y = top_left[1]
center_x = x + width
center_y = y + height
M = cv.getRotationMatrix2D((center_x, center_y), 30, 1)
rect = np.array([[x, y], [x+width*2, y], [x+width*2, y+height*2], [x, y+height*2]], dtype=np.float32)
rotated_rect = cv.transform(np.array([rect]), M)[0]
cv.polylines(rotated_query, [np.int32(rotated_rect)], True, 255, 2)

cv.imwrite('rotated_template_match_tm_ccorr.jpg', rotated_query)


True